In [1]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
r=requests.get(my_url)

In [4]:
page=bsoup(r.text,"html.parser")
#page - commented because too long as output

In [5]:
rtable=page.table
#rtable - commented because too long as output

In [6]:
results=rtable.find_all('tr')
nrows=len(results)
nrows

288

In [7]:
results[0:5]

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighborhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>]

In [8]:
header=results[0].text.split()
header

['Postcode', 'Borough', 'Neighborhood']

In [9]:
results[85].text

'\nM5J\nDowntown Toronto\nUnion Station\n'

In [10]:
results[85].text.split('\n')

['', 'M5J', 'Downtown Toronto', 'Union Station', '']

In [11]:
Postcode=results[85].text.split('\n')[1]
Postcode

'M5J'

In [12]:
Borough=results[85].text.split('\n')[2]
Borough

'Downtown Toronto'

In [13]:
Neighborhood=results[85].text.split('\n')[3]
Neighborhood

'Union Station'

In [14]:
# Iteration Loop

records =[]
n=1
while n < nrows :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[2]
    Neighborhood=results[n].text.split('\n')[3]
    records.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
df.shape

(287, 3)

In [16]:
df.tail()    #  286 because starting counting at [0]

,PostalCode,Borough,Neighbourhood
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor
286,M9Z,Not assigned,Not assigned


In [17]:
df[df['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [18]:
df1=df[~df.Borough.str.contains("Not assigned")]
df1=df1.reset_index(drop=True)

In [19]:
df1.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [20]:
df1.tail()

,PostalCode,Borough,Neighbourhood
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West
209,M8Z,Etobicoke,South of Bloor


In [21]:
df1.shape

(210, 3)

In [22]:
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']

In [23]:
df1.head()           # Cleared from 'Not assigned'

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [24]:
df1.shape

(210, 3)

In [25]:
postalcodes = df1['PostalCode'].nunique()
boroughs = df1['Borough'].nunique()
neighbourhoods= df1['Neighbourhood'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighbourhoods  :' + str(neighbourhoods))

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Neighbourhoods  :207


In [26]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [27]:
nrows1=len(df1)
nrows1

210

In [28]:
# DataFrame df2 also has 212 rows or cells
df2=df1
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [29]:
# Pointers: n goes from 0 to 211 and m from 1 to 212
nrows2=len(df2)-1
nrows2

209

In [30]:
n=0

while n < nrows2 :
    post1=df2.iloc[n,0]
    m=n+1
    post2=df2.iloc[m,0]
    neigh1=df2.iloc[n,2]
    neigh2=df2.iloc[m,2]
    if post1==post2:
        df2.Neighbourhood[n,2] = neigh1=neigh1+','+neigh2
        df2=df2.drop(df2.index[m])
        nrows2=nrows2-1
        df2 = df2.reset_index(drop=True)
    else:
        n=n+1
df2.index

RangeIndex(start=0, stop=103, step=1)

In [31]:
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [32]:
df2.index

RangeIndex(start=0, stop=103, step=1)

# The Solution of Part 1:

In [33]:
df2.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [34]:
df2.shape

(103, 3)

# Parts 2 and 3:

In [35]:
import json
from geopy.geocoders import Nominatim 

In [36]:
# ERROR: Reading the cvs file and converting it to a dataframe

#url='http://cocl.us/Geospatial_data'
#df_pcodes=pd.read_csv(url)
#df_pcodes.head()

Sorry, like other members in the forum I have the following problem: 
    
The link https://cocl.us/Geospatial_data to the coordinates for the Toronto postcodes seems to be offline. It worked earlier this week but now I just get:

"503 Service Unavailable No server is available to handle this request."